In [4]:
import json
import requests
from decimal import Decimal
import re  # Import regular expressions
import io
from rich import print as rprint
import pandas as pd
import pprint

genesis_url = 'https://www-genesis.destatis.de/genesisWS/rest/2020/data/cube?'
user_id = 'DE17T29R57'
password = '4Bf/3Ap)3]r2,,h'

In [5]:
# Preise für Leichtes Heizöl - 40 - 50 hl pro Auftrag, BERLIN
cubecode = '61241BM010'
areatype = 'all'
category = 'all'
content = ''
start_year = ''
classifier1 = 'BEORT1'
classifier2 = 'LFGAT1'
key1 = '11000000'
key2 = 'LIEFERUNGOEL02'
lang = 'en'

# Destatis Base Url
url = (f"{genesis_url}username={user_id}&password={password}&language=de&name={cubecode}&area={areatype}"
       f"&compress=true&contents={content}&startyear={start_year}")

# Add Classifyers to url
if classifier1:
       url += f'&classifyingvariable1={classifier1}'
if key1:
       url += f'&classifyingkey1={key1}'

if classifier2:
       url += f'&classifyingvariable2={classifier2}'
if key2:
       url += f'&classifyingkey2={key2}'

# request
response = requests.get(url)
# First, parse the JSON response with Decimal conversion for floats
#Float types are not supported with dynamodb; use Decimal types instead
parsed_json = json.loads(response.text, parse_float=Decimal)

# Now, access the 'Content' part and split it by newline
data = parsed_json["Object"]['Content'].split('\n')

# Compile a regex pattern to find sequences of digits ending with "=100"
info_pattern = re.compile(r'\d+=100')

# Extract only the parts of lines that match the pattern "*=100"
info_matches = []
for line in data:
    matches = info_pattern.findall(line)  # find all matches in a line
    info_matches.extend(matches)  # add found matches to the list

# Convert info_matches to a single string to append to each filtered line
info_matches_str = ', '.join(info_matches)

# Filter out the relevant lines and append the matches string to each
filtered_response = [line + " " + info_matches_str for line in data if 'D' in line and 'e' in line and ('MONAT' in line or 'QUART' in line)]

# Split each line into its components and extract the relevant information

data = [{'field_D': parts[0],
         'classifyingkey1': parts[1],
         'classifyingkey2': parts[2],
         'period': parts[3],
         'year': int(parts[4]),
         'value': float(parts[5]),
         'field_e': parts[6],
         'base': info_matches_str
        }
        for parts in (line.split(';') for line in filtered_response)]

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Determine if the data is monthly or quarterly based on the unique values in the 'period' column
if df['period'].str.contains('Q').any():
    period_key = 'quarter'
else:
    period_key = 'month'

# Create result dict
result = {'cubeCode': cubecode, 'content': content, 'classifyingVar1': classifier1, 'classifyingKey1': key1, 'classifyingVar2': classifier2, 'classifyingKey2': key2, 'data': []}

for year, group in df.groupby('year'):
    year_data = {'year': year, 'df': [{period_key: row['period'], 'value': row['value'],'Base': row['base']} for _, row in group.iterrows()]}
    result['data'].append(year_data)

rprint(json.dumps(result, indent=4))

{
    "cubeCode": "61241BM010",
    "content": "",
    "classifyingVar1": "BEORT1",
    "classifyingKey1": "11000000",
    "classifyingVar2": "LFGAT1",
    "classifyingKey2": "LIEFERUNGOEL02",
    "data": [
        {
            "year": 1978,
            "df": [
                {
                    "month": "MONAT01",
                    "value": 14.15,
                    "Base": ""
                },
                {
                    "month": "MONAT02",
                    "value": 14.13,
                    "Base": ""
                },
                {
                    "month": "MONAT03",
                    "value": 13.49,
                    "Base": ""
                },
                {
                    "month": "MONAT04",
                    "value": 13.66,
                    "Base": ""
                },
                {
                    "month": "MONAT05",
                    "value": 13.77,
                    "Base": ""
                },
                {
                    "month": "MONAT06",
                    "value": 13.62,
                    "Base": ""
                },
                {
                    "month": "MONAT07",
                    "value": 13.37,
                    "Base": ""
                },
                {
                    "month": "MONAT08",
                    "value": 13.48,
                    "Base": ""
                },
                {
                    "month": "MONAT09",
                    "value": 13.56,
                    "Base": ""
                },
                {
                    "month": "MONAT10",
                    "value": 13.6,
                    "Base": ""
                },
                {
                    "month": "MONAT11",
                    "value": 15.77,
                    "Base": ""
                },
                {
                    "month": "MONAT12",
                    "value": 16.45,
                    "Base": ""
                }
            ]
        },
        {
            "year": 1979,
            "df": [
                {
                    "month": "MONAT01",
                    "value": 18.64,
                    "Base": ""
                },
                {
                    "month": "MONAT02",
                    "value": 19.86,
                    "Base": ""
                },
                {
                    "month": "MONAT03",
                    "value": 23.74,
                    "Base": ""
                },
                {
                    "month": "MONAT04",
                    "value": 24.56,
                    "Base": ""
                },
                {
                    "month": "MONAT05",
                    "value": 24.86,
                    "Base": ""
                },
                {
                    "month": "MONAT06",
                    "value": 27.03,
                    "Base": ""
                },
                {
                    "month": "MONAT07",
                    "value": 27.61,
                    "Base": ""
                },
                {
                    "month": "MONAT08",
                    "value": 26.85,
                    "Base": ""
                },
                {
                    "month": "MONAT09",
                    "value": 26.73,
                    "Base": ""
                },
                {
                    "month": "MONAT10",
                    "value": 26.81,
                    "Base": ""
                },
                {
                    "month": "MONAT11",
                    "value": 27.88,
                    "Base": ""
                },
                {
                    "month": "MONAT12",
                    "value": 29.29,
                    "Base": ""
                }
            ]
        },
        {
            "year": 1980,
            "df": [
                {
                    "month": "MONAT01",
                    "value": 31.36,
                    "Base": 

In [ ]:
# Preise für Leichtes Heizöl - 40 - 50 hl pro Auftrag, RHEINSCHIENE 
cubecode = '61241BM010'
areatype = 'all'
category = 'all'
content = ''
start_year = ''
classifier1 = 'BEORT1'
classifier2 = 'LFGAT1'
key1 = 'RHEINSCHIENE'
key2 = 'LIEFERUNGOEL02'
lang = 'en'

# Destatis Base Url
url = (f"{genesis_url}username={user_id}&password={password}&language=de&name={cubecode}&area={areatype}"
       f"&compress=true&contents={content}&startyear={start_year}")

# Add Classifyers to url
if classifier1:
       url += f'&classifyingvariable1={classifier1}'
if key1:
       url += f'&classifyingkey1={key1}'

if classifier2:
       url += f'&classifyingvariable2={classifier2}'
if key2:
       url += f'&classifyingkey2={key2}'

# request
response = requests.get(url)
data = json.loads(response.text)["Object"]['Content'].split('\n')

# Filter out the relevant lines
filtered_response = [line for line in data if 'D' in line and 'e' in line and ('MONAT' in line or 'QUART' in line)]

# Split each line into its components and extract the relevant information

data = [{'field_D': parts[0],
         'classifyingkey1': parts[1],
         'classifyingkey2': parts[2],
         'period': parts[3],
         'year': int(parts[4]),
         'value': float(parts[5]),
         'field_e': parts[6]
        }
        for parts in (line.split(';') for line in filtered_response)]

# Create a DataFrame from the extracted data
df = pd.DataFrame(data)

# Determine if the data is monthly or quarterly based on the unique values in the 'period' column
if df['period'].str.contains('Q').any():
    period_key = 'quarter'
else:
    period_key = 'month'

# Create result dict
result = {'cubeCode': cubecode, 'content': content, 'classifyingVar1': classifier1, 'classifyingKey1': key1, 'classifyingVar2': classifier2, 'classifyingKey2': key2, 'data': []}

for year, group in df.groupby('year'):
    year_data = {'year': year, 'df': [{period_key: row['period'], 'value': row['value']} for _, row in group.iterrows()]}
    result['data'].append(year_data)

rprint(json.dumps(result, indent=4))